# 0) Binary Search

In [8]:
def binary_search(array, target):
    left, right = 0, len(array)-1

    while left <= right:
        mid = (left + right) // 2
        if target < array[mid]:
            right = mid - 1
        elif target > array[mid]:
            left = mid + 1
        else:
            return mid
    return -1

# Test
array = [1, 3, 4]
for target in [0, 1, 2, 3, 4, 5]:
    print(f'Index of {target} in array {array}: {binary_search(array, target)}')
print('\n')
array = [1, 2, 4,7]
for target in [3, 4, 7]:
    print(f'Index of {target} in array {array}: {binary_search(array, target)}')

Index of 0 in array [1, 3, 4]: -1
Index of 1 in array [1, 3, 4]: 0
Index of 2 in array [1, 3, 4]: -1
Index of 3 in array [1, 3, 4]: 1
Index of 4 in array [1, 3, 4]: 2
Index of 5 in array [1, 3, 4]: -1


Index of 3 in array [1, 2, 4, 7]: -1
Index of 4 in array [1, 2, 4, 7]: 2
Index of 7 in array [1, 2, 4, 7]: 3


# 1) Trees

*Ordered trees*: a tree is ordered where children of every node are ordered.

## Depth/Height

### Depth

The depth of a node is the number of its ancestor (excluding the node itself).

<pre>
a
├── b
├── c
    ├── d
    ├── e
</pre>  

*The depth of node d is 2.*

The depth can be computed recursively:

In [9]:
def depth(node):
    if node.isroot():
        return 0
    else:
        return 1 + depth(node.parent())

```depth()``` has a O(depth(node)) time complexity for a given node. Hence the worst-case time complexity is O(n) since a one-branch tree would yield a leaf with depth n-1.

### Height

The height of a **node** is defined as:
* height(node) = 1 + max({height(child) | child ∈ children of node}) if the node is not a leaf
* height(node) = 0 if the node is a leaf

The height of a **tree** is the height of its root. Note that height(T) = max({depth(L) | L ∈ Leaves of T})

In [10]:
def height_suboptimal(T):
    return max((depth(node) for node in T.nodes() if node.is_leaf()))

The above algorithm runs in O(n<sup>2</sup>) worst-case time: listing the nodes can be done in O(n) worst-case time and computing the depth of all the leaves has a O(n<sup>2</sup>) worst-case time complexity (see page 309 and [here](https://cs.stackexchange.com/questions/87336/maximum-sum-of-depths-of-all-external-nodes-in-a-binary-tree) for details).

Example of a tree for which computing all the leaves' depths would be n<sup>2</sup>/4 = O(n<sup>2</sup>):

<pre>
a<sub>1</sub>
├── a<sub>2</sub>
    ...
        ├── a<sub>n/2</sub>
            ├── b<sub>1</sub>
            ├── b<sub>2</sub>
            ...
            ├── b<sub>n/2</sub>
</pre>

On the other hand, using the recursive definition of the height yields a O(n) worst-case running time:

In [11]:
def height_optimal(node):
    if node.is_leaf():
        return 0
    else:
        return max((height_optimal(child) for child in node.children()))


# 2) Binary trees

A binary tree is an **ordered** tree with **at most** 2 children per node, labeled left child/right child. The order is left child then right child.

A binary tree is said to be **proper** if each node has either 0 or 2 children.

A **level** d is the set of all nodes that have a depth d. A given level d has at most 2<sup>d</sup> nodes.

## Tree Traversal Algorithms

In [12]:
### Preorder traversal
def preorder(T, p):
    T.visit(p)
    for c in T.children(p):
        preorder(T, c)

### Postorder traversal
def postorder(T, p):
    for c in T.children(p):
        postorder(T, c)
    T.visit(p)

### Breadth-first traversal
def breadthfirst(T):
    Q=Queue()
    Q.enqueue(T.root())
    while not Q.is_empty():
        p=Q.dequeue()
        p.visit()
        for c in T.children(p):
            Q.enqueue(c)

### Inorder traversal
# Specific to binary trees
def inorder(T, p):
    if p.left():
        inorder(p.left())
    p.visit()
    if p.right():
        inorder(p.right())

# 3) Binary Search Trees (BST)